# CIBERSORTx Results Analysis Primary 

In [187]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt

In [188]:
def load_fractions(dem_path):
    # load CIBERSORT output
    dat = sc.read_csv(dem_path, first_column_names=True)
    dat = dat[:,~dat.var_names.isin(['P-value', 'Correlation', 'RMSE', 'Absolute score (sig.score)'])]
    dat.obs.index.rename('Sample', inplace=True)

    # Absolute Scores
    absolute_scores = dat.to_df()
    absolute_scores.columns = absolute_scores.columns+'-Absolute'
    dat.obs = dat.obs.merge(absolute_scores, how = 'left', left_index=True, right_index=True)
    
    # Relative Scores (tumor only)
    relative_scores = dat.to_df()
    relative_scores = relative_scores.iloc[:,~relative_scores.columns.isin(['B', 'Plasma', 'T', 'CTL', 'NK', 'cDC', 'Monocyte', 'lateEry'])]
    relative_scores = (relative_scores.transpose() / relative_scores.sum(axis=1).transpose()).transpose()
    relative_scores.columns = relative_scores.columns+'-Relative'
    dat.obs = dat.obs.merge(relative_scores, how = 'left', left_index=True, right_index=True)
    
    return dat

In [190]:
PC2_loadings = hcat.varm['PCs'][:,1]
PC2_loadings = np.array([-0.20023263, -0.08772767, -0.27906924, -0.5126374, 0.7101968, 0.04527085, 0.32419923])

## Loading Prediction Output

In [191]:
def get_ranklist(dat, classification, cluster_A, cluster_B, test='wilcoxon'):
    
    adata = dat[dat.obs[classification].isin([cluster_A, cluster_B])]

    sc.tl.rank_genes_groups(adata, classification, method = test, n_genes=20000)
    sam_cat_DE_df = pd.concat(
            [pd.DataFrame(adata.uns['rank_genes_groups']['names']),
             pd.DataFrame(adata.uns['rank_genes_groups']['scores']),
             pd.DataFrame(adata.uns['rank_genes_groups']['logfoldchanges']),
             pd.DataFrame(adata.uns['rank_genes_groups']['pvals'])],
        keys = ['names','scores','logfoldchanges', 'pvals'],axis=1)

    sam_cat_DE_df.columns = sam_cat_DE_df.columns.swaplevel(0, 1)
    sam_cat_DE_df.sort_index(axis=1,level=0,inplace=True)
    sam_cat_DE_df = sam_cat_DE_df[cluster_A][['names', 'scores', 'logfoldchanges', 'pvals']]

    return(sam_cat_DE_df)

In [192]:
def get_scores(df, dat, classification, cluster_A, cluster_B, cohort, drug, target):
    # Absolute Scores
        
    ## Wilcox
    temp = get_ranklist(dat, classification, cluster_A, cluster_B, test='wilcoxon')
    temp['Cohort'] = cohort
    temp['Drug'] = drug
    temp['Target'] = target
    temp['Test'] = 'wilcox'
    temp['CellType'] = temp['names']
    temp['logPval_dir'] = np.where(temp['scores'] < 0, np.log(temp['pvals']), -np.log(temp['pvals']))
    temp['Treatment'] = np.where(temp['Cohort']=="Primary", temp['Drug'], temp['Drug'] + " [" + temp['Cohort'].str.replace('Primary ', '') + "]")
    df = df.append(temp)
    
    return(df)

# Run on Each Dataset

In [193]:
df = pd.DataFrame()

## AlCAr

In [194]:
dat = load_fractions("Data/Datasets/Primary/CIBERSORTx_Results_AlCAr_Pyrimidine.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('[0-9]','').str.replace('ctrl', 'Control')

In [195]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [196]:
df = get_scores(df, dat, 'Treatment', 'AlCAr', 'Control', 'Primary', 'AlCAr', 'Pyrimidine Synthesis')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


## 5-Aza BAY1436032 IDH

In [197]:
dat = load_fractions("Data/Datasets/Primary/CIBERSORTx_Results_Aza_IDH_IDH1mut_AML.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[-2] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = np.where(dat.obs['Treatment'] == "AZA-PRL", "5-Aza + BAY1436032", dat.obs['Treatment'])
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace("Azacitidine", "5-Aza").str.replace("PRL", "5-Aza + BAY1436032")

In [198]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [199]:
df = get_scores(df, dat, 'Treatment', 'BAY1436032', 'Control', 'Primary IDH1mut', 'BAY1436032', 'IDH')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


In [200]:
df = get_scores(df, dat, 'Treatment', '5-Aza', 'Control', 'Primary IDH1mut', '5-Aza', 'HMA')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


In [201]:
df = get_scores(df, dat, 'Treatment', '5-Aza + BAY1436032', 'Control', 'Primary IDH1mut', '5-Aza + BAY1436032', 'HMA + IDH')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


## BCL6 Fx1

In [202]:
dat = load_fractions("Data/Datasets/Primary/CIBERSORTx_Results_BCL6_TP53mut_AML.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[-2] for x in dat.obs.index.str.split('.')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

In [203]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [204]:
df = get_scores(df, dat, 'Treatment', 'Fx1', 'Control', 'Primary TP53mut', 'Fx1', 'BCL6')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


## BET CDK9 JQ1 CDKI-73

In [205]:
dat = load_fractions("Data/Datasets/Primary/CIBERSORTx_Results_BET_CDK9.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[-1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('ctl', 'Control').str.replace('CDKi', 'CDKI-73').str.replace('comb', 'JQ1 + CDKI-73')

In [206]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [207]:
df = get_scores(df, dat, 'Treatment', 'JQ1', 'Control', 'Primary MLL-r', 'JQ1', 'BET')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical
C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\scanpy\tools\_rank_genes_groups.py:407: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


In [208]:
df = get_scores(df, dat, 'Treatment', 'CDKI-73', 'Control', 'Primary MLL-r', 'CDKI-73', 'CDK9')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


In [209]:
df = get_scores(df, dat, 'Treatment', 'JQ1 + CDKI-73', 'Control', 'Primary MLL-r', 'JQ1 + CDKI-73', 'BET + CDK9')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


## HMA LSD1 5-Aza GSK2879552

In [210]:
dat = load_fractions("Data/Datasets/Primary/CIBERSORTx_Results_5aza_LSD1.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[-1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('[0-9]','')
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control').str.replace("Aza", "5-Aza").str.replace("LSDi", "GSK2879552").str.replace("GSK2879552plus5-Aza", "5-Aza + GSK2879552")

In [211]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [212]:
df = get_scores(df, dat, 'Treatment', '5-Aza', 'Control', 'Primary TET2mut', '5-Aza', 'HMA')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


In [213]:
df = get_scores(df, dat, 'Treatment', 'GSK2879552', 'Control', 'Primary TET2mut', 'GSK2879552', 'LSD1')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical
C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\scanpy\tools\_rank_genes_groups.py:407: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


In [214]:
df = get_scores(df, dat, 'Treatment', '5-Aza + GSK2879552', 'Control', 'Primary TET2mut', '5-Aza + GSK2879552', 'HMA + LSD1')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical
C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\scanpy\tools\_rank_genes_groups.py:407: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


## HMA FLT3 5-Aza Crenolanib

In [215]:
dat = load_fractions("Data/Datasets/Primary/CIBERSORTx_Results_Crenolanib_Aza_PDX.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[-2] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control').str.replace("AZA", "5-Aza").str.replace("Creno", "Crenolanib").str.replace("+", " + ")

In [216]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [217]:
df = get_scores(df, dat, 'Treatment', '5-Aza', 'Control', 'Primary FLT3-ITDmut', '5-Aza', 'HMA')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


In [218]:
df = get_scores(df, dat, 'Treatment', 'Crenolanib', 'Control', 'Primary FLT3-ITDmut', 'Crenolanib', 'FLT3')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


In [219]:
df = get_scores(df, dat, 'Treatment', '5-Aza + Crenolanib', 'Control', 'Primary FLT3-ITDmut', '5-Aza + Crenolanib', 'HMA + FLT3')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


## TRC105 CD105

In [220]:
dat = load_fractions("Data/Datasets/Primary/CIBERSORTx_Results_TRC105_Xeno.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('IgG', 'Control')

In [221]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [222]:
df = get_scores(df, dat, 'Treatment', 'TRC105', 'Control', 'Primary', 'TRC105', 'CD105')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical
C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\scanpy\tools\_rank_genes_groups.py:407: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


## XPO1 KPT

In [223]:
dat = load_fractions("Data/Datasets/Primary/CIBERSORTx_Results_XPO1_KPT.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[-1] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = np.where(dat.obs['Treatment'] == 'KPT', 'Selinexor', 'DMSO')
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

In [224]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [225]:
df = get_scores(df, dat, 'Treatment', 'Selinexor', 'Control', 'Primary', 'KPT330', 'XPO1')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


## HDAC/NAMPT AR-42 KPT-9274

In [226]:
dat = load_fractions("Data/Datasets/Primary/CIBERSORTx_HDAC8_NAMPT_Results.csv")
dat.X = (dat.X.transpose() / dat.X.sum(axis=1)).transpose()
dat.obs['Treatment'] = [x[0] for x in dat.obs.index.str.split('_')]
dat.obs['Treatment'] = dat.obs['Treatment'].str.replace('DMSO', 'Control')

In [227]:
newdat = dat.to_df()
newdat['PC2'] = (dat.to_df() * PC2_loadings).sum(axis=1)
dat = ad.AnnData(newdat, dat.obs)
del(newdat)

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [228]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'AR-42', cluster_B = 'Control', cohort = 'Primary', drug = 'AR-42', target = 'HDAC8')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


In [229]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'KPT-9274', cluster_B = 'Control', cohort = 'Primary', drug = 'KPT-9274', target = 'NAMPT')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


In [230]:
df = get_scores(df, dat, classification = 'Treatment', cluster_A = 'AR-42+KPT-9274', cluster_B = 'Control', cohort = 'Primary', drug = 'AR-42+KPT-9274', target = 'HDAC8 + NAMPT')

C:\Users\Hp\Anaconda3\envs\single_cell\lib\site-packages\anndata\_core\anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Treatment' as categorical


# Export Dataframe 

In [231]:
df.to_csv('C:/Users/Hp/Desktop/AML-drug-response/Complex Heatmap/Data/Drug_Primary_pvalues.csv')

In [232]:
df.to_csv('C:/Users/Hp/Desktop/AML-drug-response/Complex Heatmap/UMAP/Drug_Primary_pvalues.csv')